In [1]:
# Read dataset
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
# Train test split
from sklearn.model_selection import train_test_split
# Build model
from sklearn.feature_extraction.text import TfidfVectorizer
import tensorflow as tf
from keras.models import Sequential
from keras.layers import Flatten, Dense, Softmax, Dropout
from nltk.corpus import stopwords
import re

# Đọc dữ liệu

In [2]:
data = pd.read_csv('E:\Crawl_Web\Rotten Tomatoes\Preprocessing\movie_cleaned.csv')[:20000]

In [3]:
data.head(3)

,Title,Score,Review count,Genre,Rating,Synopsis
0,Paa,58.5,1008,Drama,PG-13,A politician's 13-year-old son has a rare diso...
1,Small Town Wisconsin,85.5,62,"Drama, Comedy",PG-13,"After losing a custody battle, perpetual teena..."
2,The 100-Year-Old Man Who Climbed Out the Windo...,68.5,2581,"Comedy, Adventure",R,"Allan Karlsson, a 100-year-old explosives expe..."


# Tiền xử lí dữ liệu

In [4]:
# Drop các cột không dùng trong model
data.drop(columns=['Title', 'Review count'], inplace=True)

In [5]:
data['Info'] = data['Genre'] + ' ' + data['Rating'] + ' ' + data['Synopsis']

In [6]:
# Hàm tiền xử lí dữ liệu text
def preprocess_text(text):
    text = text.lower() # Chuyển văn bản thành chữ thường
    text = re.sub(r'[^\w\s]',' ',text) # Xóa dấu câu
    text = re.sub("\d+", " ", text) # Xóa chữ số
    stop = stopwords.words('english') # Xóa các từ không có nghĩa (Stop Words)
    text = " ".join(text for text in text.split() if text not in stop)
    return text

In [7]:
data['Info'] = data['Info'].apply(preprocess_text)

In [8]:
data['Score'] = data['Score']/10

# Xây dựng mô hình

## Vector hóa dữ liệu đầu vào bằng TF-IDF

In [9]:
vectorizer = TfidfVectorizer()
features = vectorizer.fit_transform(data['Info'])

# Chọn biến X và Y

In [10]:
X = features
Y = data['Score']

# Chia tập dữ liệu thành 2 tập train(80%) và test(20%)

In [11]:
x_train, x_test, y_train, y_test = train_test_split(X, Y, test_size = 0.2, random_state = 42)

# Build model

In [12]:
model = Sequential()
model.add(Dense(64, input_dim = x_train.shape[1], activation = 'relu'))
model.add(Dropout(0.15))
model.add(Dense(64, activation = 'relu'))
model.add(Dropout(0.2))
model.add(Dense(40, activation = 'relu'))
model.add(Dropout(0.15))
model.add(Dense(54, activation = 'relu'))
model.add(Dropout(0.18))
model.add(Dense(1))

In [13]:
model.compile(optimizer = 'adam', loss = 'mean_squared_error', metrics = ['mse', 'mae'])

In [14]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 64)                3002880   
                                                                 
 dropout (Dropout)           (None, 64)                0         
                                                                 
 dense_1 (Dense)             (None, 64)                4160      
                                                                 
 dropout_1 (Dropout)         (None, 64)                0         
                                                                 
 dense_2 (Dense)             (None, 40)                2600      
                                                                 
 dropout_2 (Dropout)         (None, 40)                0         
                                                                 
 dense_3 (Dense)             (None, 54)                2

In [15]:
model.fit(x_train.todense(), y_train, epochs=20)

Epoch 1/20
500/500 [==============================] - 14s 25ms/step - loss: 8.0080 - mse: 8.0080 - mae: 2.2230
Epoch 2/20
500/500 [==============================] - 13s 26ms/step - loss: 3.4809 - mse: 3.4809 - mae: 1.4956
Epoch 3/20
500/500 [==============================] - 13s 26ms/step - loss: 2.4008 - mse: 2.4008 - mae: 1.2245
Epoch 4/20
500/500 [==============================] - 13s 26ms/step - loss: 1.7827 - mse: 1.7827 - mae: 1.0548
Epoch 5/20
500/500 [==============================] - 13s 27ms/step - loss: 1.3805 - mse: 1.3805 - mae: 0.9246
Epoch 6/20
500/500 [==============================] - 13s 27ms/step - loss: 1.1789 - mse: 1.1789 - mae: 0.8542
Epoch 7/20
500/500 [==============================] - 14s 27ms/step - loss: 0.9993 - mse: 0.9993 - mae: 0.7873
Epoch 8/20
500/500 [==============================] - 14s 27ms/step - loss: 0.8981 - mse: 0.8981 - mae: 0.7449
Epoch 9/20
500/500 [==============================] - 13s 27ms/step - loss: 0.8027 - mse: 0.8027 - mae: 0.7046
E

# Dự đoán điểm phim

In [17]:
# Thể loại phim
genre = 'Sci-fi, Adventure, Action, Fantasy, Comedy'

# Rating phim
rating = 'PG-13'


# Tóm tắt phim
synopsis = 'In Marvel Studios "Guardians of the Galaxy Vol. 3" our beloved band of misfits are looking a bit different these days. Peter Quill, still reeling from the loss of Gamora, must rally his team around him to defend the universe along with protecting one of their own. A mission that, if not completed successfully, could quite possibly lead to the end of the Guardians as we know them.'

In [22]:
# Dự đoán
text = genre + ' ' + rating + ' ' + synopsis
info_vector = vectorizer.transform([preprocess_text(text)])

predict = model.predict(info_vector.todense())
print(predict)

1/1 [==============================] - 0s 19ms/step
[[7.1346207]]
